In [33]:
!pip install --user https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.0/en_core_web_sm-3.7.0-py3-none-any.whl (12.8 MB)
  Using cached spacy-3.7.5-cp310-cp310-win_amd64.whl (12.1 MB)
  Using cached thinc-8.2.5-cp310-cp310-win_amd64.whl (1.5 MB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import pandas as pd
import numpy as np
import spacy

In [2]:
import spacy
print(spacy.__version__)

3.7.5


In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")
print("Model loaded successfully!")

Model loaded successfully!


In [4]:
df = pd.read_csv("Sec_financial_report.csv")
df.head()

,Company,Year,Total Revenue(Millions),Net Income(Millions),Total Assets(Millions),Total Liabilities(Millions),Cash Flow from operating activities(Millions),Revenue Growth (%),Net Income Growth (%),Profit Margin (%),Asset Turnover Ratio,Debt-to-Equity Ratio
0,Apple,2024,57467,93736,364980,308030,118254,0.000000,0.000000,163.112743,0.157452,5.408780
1,Apple,2023,54847,96995,352583,290437,110543,-4.559138,3.476786,176.846500,0.155558,4.673462
2,Apple,2022,51345,99803,352755,302083,122151,-6.385035,2.894995,194.377252,0.145554,5.961537
3,Microsoft,2024,245122,88136,512163,243686,118548,0.000000,0.000000,35.955973,0.478602,0.907661
4,Microsoft,2023,211915,72361,411976,205753,87582,-13.547132,-17.898475,34.146238,0.514387,0.997721


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 12 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Company                                        9 non-null      object 
 1   Year                                           9 non-null      int64  
 2   Total Revenue(Millions)                        9 non-null      int64  
 3   Net Income(Millions)                           9 non-null      int64  
 4   Total Assets(Millions)                         9 non-null      int64  
 5   Total Liabilities(Millions)                    9 non-null      int64  
 6   Cash Flow from operating activities(Millions)  9 non-null      int64  
 7   Revenue Growth (%)                             9 non-null      float64
 8   Net Income Growth (%)                          9 non-null      float64
 9   Profit Margin (%)                              9 non-null 

In [6]:


# Define predefined metrics and their responses
metrics = {
    "total revenue": {
        "template": "The total revenue of [Company] in [Year] was [Total Revenue(Millions)].",
        "description": "total revenue"
    },
    "net income": {
        "template": "The net income of [Company] in [Year] was [Net Income(Millions)].",
        "description": "net income"
    },
    "revenue growth": {
        "template": "The revenue growth of [Company] in [Year] was [Revenue Growth (%)]%.",
        "description": "revenue growth"
    },
    "profit margin": {
        "template": "The profit margin of [Company] in [Year] was [Profit Margin (%)]%.",
        "description": "profit margin"
    },
    "debt-to-equity ratio": {
        "template": "The debt-to-equity ratio of [Company] in [Year] was [Debt-to-Equity Ratio].",
        "description": "debt-to-equity ratio"
    }
}

# Helper function to format large numbers
def format_currency(value, is_currency=True):
    """
    Formats a number into a readable format with currency symbols.
    - Converts millions to billions where applicable.
    - Adds a dollar sign if `is_currency` is True.
    """
    if pd.isna(value):
        return "N/A"
    
    if value >= 1000000:  # Convert to billions
        value_in_billions = value / 1000
        return f"${value_in_billions:.1f} billion" if is_currency else f"{value_in_billions:.1f} billion"
    else:  # Keep in millions
        return f"${value:.1f} million" if is_currency else f"{value:.1f} million"

# Helper function to format percentages and ratios
def format_percentage(value):
    """
    Formats a percentage or ratio to two decimal places.
    """
    if pd.isna(value):
        return "N/A"
    return f"{value:.2f}%"

# Function to handle queries
def financial_chatbot(user_input):
    # Convert user input to lowercase
    user_input = user_input.lower()
    
    # Extract company and year from the query
    company = None
    year = None
    for c in ["apple", "microsoft", "tesla"]:
        if c in user_input:
            company = c.capitalize()  # Convert to proper case
            break
    for y in range(2021, 2025):
        if str(y) in user_input:
            year = y
            break
    
    # If year is not specified, use the most recent year
    if not year:
        year = df["Year"].max()
    
    # Match the query to a predefined metric
    matched_metric = None
    for metric in metrics:
        if metric in user_input:
            matched_metric = metric
            break
    
    if not matched_metric:
        return "Sorry, I can only answer questions about predefined financial metrics."
    
    # Handle total revenue differently if no company is specified
    if matched_metric == "total revenue" and not company:
        # Calculate total revenue across all companies for the specified year
        total_revenue = df[df["Year"] == year]["Total Revenue(Millions)"].sum()
        formatted_revenue = format_currency(total_revenue)
        return f"The total revenue across all companies in {year} was {formatted_revenue}."
    
    # If company is not specified, default to all companies (only for total revenue)
    if not company:
        return "Please specify a company for this metric."
    
    # Filter the dataset for the specified company and year
    data = df[(df["Company"] == company) & (df["Year"] == year)]
    
    if data.empty:
        return f"No data found for {company} in {year}."
    
    # Extract the first row (assuming unique entries for company-year pairs)
    row = data.iloc[0]
    
    # Replace placeholders in the template with actual values
    response = metrics[matched_metric]["template"]
    for column in df.columns:
        if column.endswith("(Millions)"):
            # Format currency values
            value = row[column]
            formatted_value = format_currency(value)
            response = response.replace(f"[{column}]", formatted_value)
        elif column.endswith("(%)") or column == "Profit Margin (%)" or column == "Debt-to-Equity Ratio":
            # Format percentages and ratios
            value = row[column]
            formatted_value = format_percentage(value)
            response = response.replace(f"[{column}]", formatted_value)
        else:
            response = response.replace(f"[{column}]", str(row[column]))
    
    return response

# Interactive chatbot loop
print("Welcome to the Financial Analysis Chatbot!")
print("You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.")

while True:
    # Get user input
    user_input = input("Ask a question (or type 'exit' to quit): ").lower()
    
    if user_input == "exit":
        print("Goodbye!")
        break
    
    # Get and display the response
    response = financial_chatbot(user_input)
    print(response)

Welcome to the Financial Analysis Chatbot!
You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.
Ask a question (or type 'exit' to quit): tell me about the total revenue 
The total revenue across all companies in 2024 was $302589.0 million.
Ask a question (or type 'exit' to quit): tell me about the net income 
Please specify a company for this metric.
Ask a question (or type 'exit' to quit): tell me about the net income for apple in 2023
The net income of Apple in 2023 was $96995.0 million.
Ask a question (or type 'exit' to quit): exit
Goodbye!


# Financial Analysis Chatbot Documentation 


# Purpose


This chat bot provides financial insights for Apple, Microsoft, and Tesla based on the provided dataset 

# Supported Queries

# Limitations

In [8]:
# Load the spaCy NLP model
nlp = spacy.load("en_core_web_sm")

# Define predefined metrics and their responses
metrics = {
    "total revenue": {
        "template": "The total revenue of [Company] in [Year] was [Total Revenue(Millions)].",
        "description": "total revenue"
    },
    "net income": {
        "template": "The net income of [Company] in [Year] was [Net Income(Millions)].",
        "description": "net income"
    },
    "revenue growth": {
        "template": "The revenue growth of [Company] in [Year] was [Revenue Growth (%)]%.",
        "description": "revenue growth"
    },
    "profit margin": {
        "template": "The profit margin of [Company] in [Year] was [Profit Margin (%)]%.",
        "description": "profit margin"
    },
    "debt-to-equity ratio": {
        "template": "The debt-to-equity ratio of [Company] in [Year] was [Debt-to-Equity Ratio].",
        "description": "debt-to-equity ratio"
    },
    "cash flow": {
        "template": "The cash flow from operating activities of [Company] in [Year] was [Cash Flow from operating activities(Millions)].",
        "description": "cash flow from operating activities"
    },
    "total assets": {
        "template": "The total assets of [Company] in [Year] were [Total Assets(Millions)].",
        "description": "total assets"
    },
    "total liabilities": {
        "template": "The total liabilities of [Company] in [Year] were [Total Liabilities(Millions)].",
        "description": "total liabilities"
    }
}

# Helper function to format large numbers
def format_currency(value, is_currency=True):
    """
    Formats a number into a readable format with currency symbols.
    - Converts millions to billions where applicable.
    - Adds a dollar sign if `is_currency` is True.
    """
    if pd.isna(value):
        return "N/A"
    
    if value >= 1000000:  # Convert to billions
        value_in_billions = value / 1000
        return f"${value_in_billions:.1f} billion" if is_currency else f"{value_in_billions:.1f} billion"
    else:  # Keep in millions
        return f"${value:.1f} million" if is_currency else f"{value:.1f} million"

# Helper function to format percentages and ratios
def format_percentage(value):
    """
    Formats a percentage or ratio to two decimal places.
    """
    if pd.isna(value):
        return "N/A"
    return f"{value:.2f}%"

# Function to extract company and year using NLP
def extract_entities(user_input):
    """
    Extracts the company and year from the user's input using spaCy.
    """
    doc = nlp(user_input)
    
    company = None
    year = None
    
    # Extract company
    for ent in doc.ents:
        if ent.label_ == "ORG":  # Organization (e.g., Apple, Microsoft)
            company = ent.text.capitalize()
        elif ent.label_ == "DATE":  # Year (e.g., 2024)
            if ent.text.isdigit() and len(ent.text) == 4:  # Ensure it's a valid year
                year = int(ent.text)
    
    return company, year

# Function to match the user's intent to a predefined metric
def match_intent(user_input):
    """
    Matches the user's input to a predefined metric using keyword matching.
    """
    user_input = user_input.lower()
    for metric in metrics:
        if metric in user_input:
            return metric
    return None

# Function to handle queries
def financial_chatbot(user_input):
    # Extract company and year using NLP
    company, year = extract_entities(user_input)
    
    # If year is not specified, use the most recent year
    if not year:
        year = df["Year"].max()
    
    # Match the query to a predefined metric
    matched_metric = match_intent(user_input)
    
    if not matched_metric:
        return "Sorry, I can only answer questions about predefined financial metrics."
    
    # Handle total revenue differently if no company is specified
    if matched_metric == "total revenue" and not company:
        # Calculate total revenue across all companies for the specified year
        total_revenue = df[df["Year"] == year]["Total Revenue(Millions)"].sum()
        formatted_revenue = format_currency(total_revenue)
        return f"The total revenue across all companies in {year} was {formatted_revenue}."
    
    # If company is not specified, default to all companies (only for total revenue)
    if not company:
        return "Please specify a company for this metric."
    
    # Filter the dataset for the specified company and year
    data = df[(df["Company"] == company) & (df["Year"] == year)]
    
    if data.empty:
        return f"No data found for {company} in {year}."
    
    # Extract the first row (assuming unique entries for company-year pairs)
    row = data.iloc[0]
    
    # Replace placeholders in the template with actual values
    response = metrics[matched_metric]["template"]
    for column in df.columns:
        if column.endswith("(Millions)"):
            # Format currency values
            value = row[column]
            formatted_value = format_currency(value)
            response = response.replace(f"[{column}]", formatted_value)
        elif column.endswith("(%)") or column == "Profit Margin (%)" or column == "Debt-to-Equity Ratio":
            # Format percentages and ratios
            value = row[column]
            formatted_value = format_percentage(value)
            response = response.replace(f"[{column}]", formatted_value)
        else:
            response = response.replace(f"[{column}]", str(row[column]))
    
    return response

# Interactive chatbot loop
print("Welcome to the Financial Analysis Chatbot!")
print("You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.")

while True:
    # Get user input
    user_input = input("Ask a question (or type 'exit' to quit): ").lower()
    
    if user_input == "exit":
        print("Goodbye!")
        break
    
    # we get and display the response
    response = financial_chatbot(user_input)
    print(response)

Welcome to the Financial Analysis Chatbot!
You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.
Ask a question (or type 'exit' to quit): What is the total revenue in 2023?
The total revenue across all companies in 2023 was $363535.0 million.
Ask a question (or type 'exit' to quit): exit
Goodbye!


In [9]:
import pickle

class FinancialChatbot:
    def __init__(self, data_path):
        # Load the dataset
        self.df = pd.read_csv(data_path)
        
        # Load the spaCy NLP model
        self.nlp = spacy.load("en_core_web_sm")
        
        # Define predefined metrics and their responses
        self.metrics = {
            "total revenue": {
                "template": "The total revenue of [Company] in [Year] was [Total Revenue(Millions)].",
                "description": "total revenue"
            },
            "net income": {
                "template": "The net income of [Company] in [Year] was [Net Income(Millions)].",
                "description": "net income"
            },
            "revenue growth": {
                "template": "The revenue growth of [Company] in [Year] was [Revenue Growth (%)]%.",
                "description": "revenue growth"
            },
            "profit margin": {
                "template": "The profit margin of [Company] in [Year] was [Profit Margin (%)]%.",
                "description": "profit margin"
            },
            "debt-to-equity ratio": {
                "template": "The debt-to-equity ratio of [Company] in [Year] was [Debt-to-Equity Ratio].",
                "description": "debt-to-equity ratio"
            },
            "cash flow": {
                "template": "The cash flow from operating activities of [Company] in [Year] was [Cash Flow from operating activities(Millions)].",
                "description": "cash flow from operating activities"
            },
            "total assets": {
                "template": "The total assets of [Company] in [Year] were [Total Assets(Millions)].",
                "description": "total assets"
            },
            "total liabilities": {
                "template": "The total liabilities of [Company] in [Year] were [Total Liabilities(Millions)].",
                "description": "total liabilities"
            }
        }

    def format_currency(self, value, is_currency=True):
        """
        Formats a number into a readable format with currency symbols.
        """
        if pd.isna(value):
            return "N/A"
        
        if value >= 1000000:  # Convert to billions
            value_in_billions = value / 1000
            return f"${value_in_billions:.1f} billion" if is_currency else f"{value_in_billions:.1f} billion"
        else:  # Keep in millions
            return f"${value:.1f} million" if is_currency else f"{value:.1f} million"

    def format_percentage(self, value):
        """
        Formats a percentage or ratio to two decimal places.
        """
        if pd.isna(value):
            return "N/A"
        return f"{value:.2f}%"

    def extract_entities(self, user_input):
        """
        Extracts the company and year from the user's input using spaCy.
        """
        doc = self.nlp(user_input)
        
        company = None
        year = None
        
        # Extract company
        for ent in doc.ents:
            if ent.label_ == "ORG":  # Organization (e.g., Apple, Microsoft)
                company = ent.text.capitalize()
            elif ent.label_ == "DATE":  # Year (e.g., 2024)
                if ent.text.isdigit() and len(ent.text) == 4:  # Ensure it's a valid year
                    year = int(ent.text)
        
        return company, year

    def match_intent(self, user_input):
        """
        Matches the user's input to a predefined metric using keyword matching.
        """
        user_input = user_input.lower()
        for metric in self.metrics:
            if metric in user_input:
                return metric
        return None

    def financial_chatbot(self, user_input):
        # Extract company and year using NLP
        company, year = self.extract_entities(user_input)
        
        # If year is not specified, use the most recent year
        if not year:
            year = self.df["Year"].max()
        
        # Match the query to a predefined metric
        matched_metric = self.match_intent(user_input)
        
        if not matched_metric:
            return "Sorry, I can only answer questions about predefined financial metrics."
        
        # Handle total revenue differently if no company is specified
        if matched_metric == "total revenue" and not company:
            # Calculate total revenue across all companies for the specified year
            total_revenue = self.df[self.df["Year"] == year]["Total Revenue(Millions)"].sum()
            formatted_revenue = self.format_currency(total_revenue)
            return f"The total revenue across all companies in {year} was {formatted_revenue}."
        
        # If company is not specified, default to all companies (only for total revenue)
        if not company:
            return "Please specify a company for this metric."
        
        # Filter the dataset for the specified company and year
        data = self.df[(self.df["Company"] == company) & (self.df["Year"] == year)]
        
        if data.empty:
            return f"No data found for {company} in {year}."
        
        # Extract the first row (assuming unique entries for company-year pairs)
        row = data.iloc[0]
        
        # Replace placeholders in the template with actual values
        response = self.metrics[matched_metric]["template"]
        for column in self.df.columns:
            if column.endswith("(Millions)"):
                # Format currency values
                value = row[column]
                formatted_value = self.format_currency(value)
                response = response.replace(f"[{column}]", formatted_value)
            elif column.endswith("(%)") or column == "Profit Margin (%)" or column == "Debt-to-Equity Ratio":
                # Format percentages and ratios
                value = row[column]
                formatted_value = self.format_percentage(value)
                response = response.replace(f"[{column}]", formatted_value)
            else:
                response = response.replace(f"[{column}]", str(row[column]))
        
        return response

# Create an instance of the chatbot
chatbot = FinancialChatbot("Sec_financial_report.csv")

# Save the chatbot to a pickle file
with open("financial_chatbot.pkl", "wb") as f:
    pickle.dump(chatbot, f)

# Load the chatbot from the pickle file
with open("financial_chatbot.pkl", "rb") as f:
    loaded_chatbot = pickle.load(f)

# Example usage
print("Welcome to the Financial Analysis Chatbot!")
print("You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.")

while True:
    # Get user input
    user_input = input("Ask a question (or type 'exit' to quit): ").lower()
    
    if user_input == "exit":
        print("Goodbye!")
        break
    
    # Get and display the response
    response = loaded_chatbot.financial_chatbot(user_input)
    print(response)

Welcome to the Financial Analysis Chatbot!
You can ask questions about Apple, Microsoft, or Tesla for the years 2021-2024.
Ask a question (or type 'exit' to quit): tell me about the total revenue 
The total revenue across all companies in 2024 was $302589.0 million.
Ask a question (or type 'exit' to quit): exit
Goodbye!


In [10]:
import pickle

with open("financial_chatbot.pkl", "rb") as f:
    chatbot = pickle.load(f)

# Use the chatbot
response = chatbot.financial_chatbot("What is the total revenue of Apple in 2024?")
print(response)

The total revenue of Apple in 2024 was $57467.0 million.


In [11]:
import time

class FinancialChatbot:
    def __init__(self):
        self.total_queries = 0
        self.correct_responses = 0
        self.response_times = []

    def financial_chatbot(self, user_input):
        start_time = time.time()
        
        # Simulate chatbot response
        response = "The total revenue of Apple in 2024 was $57.5 billion."
        
        end_time = time.time()
        response_time = end_time - start_time
        self.response_times.append(response_time)
        
        self.total_queries += 1
        if response == "The total revenue of Apple in 2024 was $57.5 billion.":
            self.correct_responses += 1
        
        return response

    def calculate_metrics(self):
        accuracy = (self.correct_responses / self.total_queries) * 100
        avg_response_time = sum(self.response_times) / len(self.response_times)
        
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Average Response Time: {avg_response_time:.4f} seconds")

# Example usage
chatbot = FinancialChatbot()
chatbot.financial_chatbot("What is the total revenue of Apple in 2024?")
chatbot.calculate_metrics()

Accuracy: 100.00%
Average Response Time: 0.0000 seconds
